# Gestión Moderna de Portafolio
### Autores Bernardo León y Carlos Zapata
### (C) Copyright 2023

## Capitulo 6: Medida Omega $\Omega$


In [1]:
#Librerías usadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pip install --quiet yfinance
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 6.7 MB/s eta 0:00:00


### Ejemplo 6.1

In [2]:
# Información histórica para las acciones: AAPL, AMZN, GOOG y MSFT
fechai = "2015-12-01"
fechaf = "2020-12-31"
periodicidad = "1Mo"
activos = ["AAPL","AMZN","GOOGL","MSFT"]
precios = yf.download(activos,start=fechai,end=fechaf,interval=periodicidad)['Adj Close'].dropna()
retornos = np.log(precios/precios.shift(1)).dropna()
mu = retornos.mean()*12
cov = retornos.cov()*12
var = np.diag(cov)
sigma = np.sqrt(var)
n = len(mu)

[*********************100%***********************]  4 of 4 completed


In [3]:
# Calculo excesos y umbral h
h = 0
excesos = retornos-h
pos_ret = excesos[excesos>h].sum()
neg_ret = excesos[excesos<h].sum()
#pos_ret, neg_ret 
omegai = pos_ret / (-neg_ret )
omegai

AAPL     2.217805
AMZN     2.314771
GOOGL    1.734004
MSFT     3.257870
dtype: float64

# Optimización usando GLPK y CVXPY

In [204]:
import cvxpy as cp

def omega(retornos, h):
    s = retornos.shape[0]
    n = retornos.shape[1]
    mu = retornos.mean(axis=0)
    cov = retornos.cov()
    y = cp.Variable(n)
    u = cp.Variable(s)
    z = cp.Variable()
    constraints = [cp.matmul(retornos, y) + u >= h,
                   cp.sum(y) == z,
                   cp.sum(u) == 1,
                   cp.matmul(mu, y) >= h*z,
                   z >= 1e-05]
    objective = cp.Minimize(cp.matmul(mu, y) - h*z)
    #objective = cp.Maximize(mu @ y - h * z)
    return objective, constraints

def budget_constraint(retornos, dir = "==", rhs = 1):
    x = cp.Variable(retornos.shape[1])
    constraints = [cp.sum(x) == rhs]
    prob = cp.Problem(cp.Minimize(0), constraints)
    prob.solve(solver=cp.GLPK)
    return x.value

wpomega = omega(retornos, h)
wpomega
#prob.variables()[0].value